In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
# load the metadata
metadata_df = pd.read_csv("metadata.csv")
title_embeddings = np.load("metadata_title_embeddings.npz")["embeddings"]
keywords_embeddings = np.load("metadata_keywords_embeddings.npz")["embeddings"]

In [3]:
# Setting index
metadata_df.set_index("filename", inplace=True)
metadata_df.head(2)


,keywords,freesound_id,sound_link,manufacturer,license,title,description,num_downloads,geotag,start_time_s,end_time_s
filename,,,,,,,,,,,
617030.mp3,"atmosphere, general-noise, suburban, ambiance,...",617030,https://freesound.org/people/klankbeeld/sounds...,klankbeeld,https://creativecommons.org/licenses/by/4.0/,suburb night NL 1041PM 210415_0279.wav,Night in a suburban city. All sounds far. Tra...,65,51.7090740261 5.30657821347,15.6,43.535
637408.mp3,"jackhammer, construction, reflected",637408,https://freesound.org/people/kyles/sounds/637408/,kyles,http://creativecommons.org/publicdomain/zero/1.0/,construction jackhammer reflected5.flac,construction jackhammer reflected5\n\nrecorded...,23,NaN,209.8,232.356


## metadata_df General look

In [4]:
##  Shape of Data Frame
print("    Shape of Data Frame:")
print("DataFrame shape: \t", metadata_df.shape)
print("DataFrame rows: \t", len(metadata_df.index))
print("DataFrame columns: \t", len(metadata_df.columns))
print("DataFrame column names: \n", metadata_df.columns.to_list())

# # select a column
# metadata['keywords']
# metadata.loc[:, 'keywords']
# metadata[['title', 'keywords']]

# # select a row
# metadata.loc['450016.mp3']
# metadata.loc[['438858.mp3', '128446.mp3'], ["title", "manufacturer"]]

#select a row based on a condition
# (metadata['manufacturer'] == 'FullMetalJedi').head(n=3)


    Shape of Data Frame:
DataFrame shape: 	 (9026, 11)
DataFrame rows: 	 9026
DataFrame columns: 	 11
DataFrame column names: 
 ['keywords', 'freesound_id', 'sound_link', 'manufacturer', 'license', 'title', 'description', 'num_downloads', 'geotag', 'start_time_s', 'end_time_s']


## Annotations file

In [5]:
annotations = pd.read_csv("annotations.csv")
# making duration column
# annotations["duartion"] = annotations["offset"] - annotations["onset"]
annotations.head(1)


,task_id,filename,annotator,text,onset,offset,filename_unsafe
0,161977861,560761.mp3,5945971035380930099053858595454362549806990533...,bird twitters nearby with a high pitch two times,16.552368,17.271435,560761_tufted titmouse calling.mp3


In [6]:
print("    Shape of annotation:")
print("DataFrame shape: \t", annotations.shape)
print("DataFrame rows: \t", len(annotations.index))
print("DataFrame columns: \t", len(annotations.columns))
print("DataFrame column names: \n", annotations.columns.to_list())

# num annotaors for each audio recording
num_annotators = annotations.groupby("filename")["annotator"].nunique()
print(f'\n number of Annotators (each one annotatd many times): {len(num_annotators)}')  

    Shape of annotation:
DataFrame shape: 	 (35826, 7)
DataFrame rows: 	 35826
DataFrame columns: 	 7
DataFrame column names: 
 ['task_id', 'filename', 'annotator', 'text', 'onset', 'offset', 'filename_unsafe']

 number of Annotators (each one annotatd many times): 9026


In [7]:
# # filtering files with more than 1 annotator
# multi_annotators = num_annotators[num_annotators > 1]
# print(f'number of Annotators (each one annotatd many times): {len(multi_annotators)}')

# # Filter metadata_df and annotations using multi_annotators
# filtered_metadata = metadata_df.loc[multi_annotators]
# filtered_annotations = annotations[annotations["filename"].isin(multi_annotators)]






## 2. Annotation Quality

## Use the complete data set (or a subset) to address the following points quantitatively.

(a) How many annotations did we collect per file? \
    How many distinct sound events per file?

In [8]:
# annotation count per task/file
annotation_counts = annotations["task_id"].value_counts()
print("Min: ", annotation_counts.min())
print("Max: ", annotation_counts.max())
print("Mean: ", annotation_counts.mean())
print("Median: ", annotation_counts.median())



# ANSWER
"""- max annotation count is 96 per file, min is 1, the mean is ~ 4,
 But most files have 2-7 annotations (more density in this range).
- I took a look at 10 samples... --> most events had 1-4 different sound events."""

Min:  1
Max:  96
Mean:  3.9692000886328387
Median:  2.0


'- max annotation count is 96 per file, min is 1, the mean is ~ 4,\n But most files have 2-7 annotations (more density in this range).\n- I took a look at 10 samples... --> most events had 1-4 different sound events.'

##  (b) How detailed are the text annotations? How much does the quality of annotations vary between different annotators?

In [ ]:
"""
Took 10 samples.
Created 10 lists of annotation of 10 different files.
"""

# first group the data by filename
grouped_by_filename = annotations.groupby("filename")

# get 10 random filenames from the grouped data
subset_filenames = list(grouped_by_filename.groups.keys())[100:110]

#loop over each group of each file and get the annotations in lists
annotations_list = []
for filename in subset_filenames:
    annotations_list.append(grouped_by_filename.get_group(filename)["text"].to_list())
# print the annotations for each file
for i, filename in enumerate(subset_filenames):
    print(f"Annotations for {filename}:")
    for j, annotation in enumerate(annotations_list[i]):
        print(f"{j + 1}: {annotation}")
    print("\n")


Annotations for 109434.mp3:
1: A loud police-siren goes on and off in a rhythmic pattern neither fading nor increasing.


Annotations for 109793.mp3:
1: A string instrument gets very deep note plucked with long decay


Annotations for 110129.mp3:
1: sound of a bus engine


Annotations for 110133.mp3:
1: continuous alarm of a cardreader next to a door, repeated sound with 500 milliseconds beep and 500 ms pause


Annotations for 110210.mp3:
1: A person close-by laughing in a lobby.
2: A baby repeatedly and faintly crying in a lobby
3: A baby faintly repeatedly crying in a lobby
4: A clear and very loud cry from a child in a lowly crouded lobby.


Annotations for 110219.mp3:
1: brass band playing continuously
2: brass band playing continuously


Annotations for 110260.mp3:
1: Rhythmic trumpets are heard clearly, their bright, brassy sound cutting through the air with a lively and energetic tune.
2: Rhythmic trumpets are heard clearly, their bright, brassy sound cutting through the air wit

In [ ]:
second_file = annotations.loc["127606.mp3"]
num_annotators = second_file["annotator"].nunique()

print(f'WE have ({len(second_file)}) annotations from ({num_annotators}) annotators for 127606.mp3')

num_annotators = second_file["annotator"].nunique()
print(f'WE have ({num_annotators}) annotators for 127606.mp3')

annotations_2nd_file = second_file["text"].to_list()
for ann in annotations_2nd_file: #print the annotations
    print(ann)

second_file.head(1)


In [ ]:
plt.title("Histogram of anotations Counts")
annotation_counts.plot.hist(bins=annotation_counts.max(), figsize=(7, 4))
plt.xlabel("annotation counts")
plt.show()
# # another plot with zoom on x-axis
# plt.title("ZOOM Histogram of anotations Counts")
# annotation_counts.plot.hist(bins=annotation_counts.max(), figsize=(7, 4))
# plt.xlim(0, 20)
# plt.xlabel("annotation counts")
# plt.show()


In [ ]:
# group by number of annotations that each user did for each task/file
annotations_per_user = annotations.groupby("annotator")["task_id"].count()
annotations_per_user 
# plot
plt.title("Histogram of Annotations per User")
annotations_per_user.plot.hist(bins=annotations_per_user.max(), figsize=(7, 4))
plt.xlabel("annotations per user")
plt.show()


In [ ]:
annotations = pd.read_csv("annotations.csv")
annotations[["filename","text"]].head(20)

In [ ]:
duplicates = annotations.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")
annotations[duplicates].head(10)


In [ ]:
print(annotations.info())
print(annotations.describe())

## Annotattion Quality
\
(a) How precise are the temporal annotations?

- The annotations are precise in generall

(b) How similar are the text annotations that correspond to the same region?

- Both files' annotations indicate the same meaning for the corresponding parts of audio.

 Use the complete data set (or a subset) to address the following points quantitatively.

(a) How many annotations did we collect per file? How many distinct sound events per file?

- max annotation count is 96 per file, min is 1, the mean is 4, But most files have 2-7 annotations (more density in this range).
- I took a look at 10 samples... --> most events had 1-4 different sound events.

 (b) How detailed are the text annotations? How much does the quality of annotations vary between
 different annotators?

 - Most text annotations have adjectives describing the sound and the closeness of it (background, in distance....) whic follows the annoting criteria.
 - In General, annotaters gave good & comlete sentences as spesified in task 1. There were, of course, some annotations with lower quality (less describing words) but still correct description. 

 (c) Are there any obvious inconsistencies, outliers, or poor-quality annotations in the data? Propose a simple method to filter or fix incorrect or poor-quality annotations (e.g., remove outliers, typos, or
 spelling errors).

 - The data seems to be clean with no outliers or poor-quality annotations
 - Checking the semilarity of texts require a method to check if some sentences have same meaning
 - To fix that , we could remove null values/ typos /duplicats, remove oset at 29 because this is the end of audio